In [110]:
from urllib.request import Request, urlopen as uReq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import io
import json
import os
import pandas as pd
from config import conn_host, conn_database, conn_user, conn_password
from IPython.display import clear_output

In [104]:
league = "Premier-League"
start_season = 2017
end_season = 2023

In [105]:
option = Options()
# option.headless = True
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# driver.maximize_window()

C:\Users\caiog\AppData\Local\Temp\ipykernel_14888\3468875481.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [106]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
driver.get(url)

driver.maximize_window()

fb = driver.find_element(By.CLASS_NAME, 'fb')
rows = fb.find_elements(By.XPATH, '//table/tbody/tr')

In [107]:
seasons = dict()

for season in range(start_season, end_season):
    seasons[season] = []
    
    url = f"https://fbref.com/en/comps/9/{season}-{season+1}/schedule/{league}-Scores-and-Fixtures"
    driver.get(url)

    driver.maximize_window()

    fb = driver.find_element(By.CLASS_NAME, 'fb')
    rows = fb.find_elements(By.XPATH, '//table/tbody/tr')

    total_games = 0
    for i, r in enumerate(rows):
        clear_output()
        print(f"{season}/{end_season-1} {i}/{len(rows)}")
        if not r.text: continue
        tds = r.find_elements(By.XPATH, './/child::td')
        week = r.find_element(By.XPATH, './/child::th').text
        if len(tds) < 13: continue
        _, date, _, home_team, home_xg, score, away_xg, away_team, _, _, _, _, _ = [t.text for t in tds]
        if not score: continue
        home_score, away_score = score.split('–')
        match_info = [date, week, home_team, home_xg, home_score, away_score, away_xg, away_team]
        match_str = f"{date} ({week}): {home_team} ({home_xg}) {home_score} x {away_score} ({away_xg}) {away_team}"
        seasons[season].append(match_info)
#         print(match_str)
        total_games += 1

2022/2022 479/480


In [112]:
directory = f"./leagues/{league.lower()}"
if not os.path.exists(directory):
    os.makedirs(directory)

for season in seasons.keys():
    print(f"Total games in the {season} season:", len(seasons[season]))
    columns = ['date', 'week', 'home_team', 'home_xg', 'home_score', 'away_score', 'away_xg', 'away_team']
    season_df = pd.DataFrame(seasons[season], columns=columns)
    season_df.to_csv(f"{directory}/{season}-{season+1}.csv")

Total games in the 2017 season: 380
Total games in the 2018 season: 380
Total games in the 2019 season: 380
Total games in the 2020 season: 380
Total games in the 2021 season: 380
Total games in the 2022 season: 367


In [113]:
driver.close()